# Task Manager

- Task manager stores user information in the users.json file to operate for accessing users and updating users fast.
  ` {"1": {"User Name": 'user1', "Password": '<string hashed password>'},}`
- It also tasks in separate tasks.json file. format like {user_id: {task_id: {task data}}}
  `{"1": {"1": {"Task Id": '1', "status": 'pending', "Description": 'Task1 by user 1'},},}`

In [267]:
# package for password hashing
import bcrypt
import base64

# json file handler package
import json


### Utility Functions.
- Read and write functionality for json file.
- json files going to store user data and task data

In [269]:

# Function to write user information to a JSON file
def write_to_json(filename, data):
    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4) 
        print(f"Data has been written to {filename}")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

# Function to read user information from a JSON file
def read_from_json(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)  # Read data from JSON file
        return data
    except FileNotFoundError:
        # If there is no existing file  return empty dict so that it can be handled while writing.
        return {}
    except json.JSONDecodeError:
        print("Error decoding JSON. The file might be corrupted.")
        return []
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return {}

In [272]:
# Utility function to display tasks header in table format
def print_header(columns):
    """
    The utility function to print table header in a nice format.
    --------------------------------------------------------------------------------
    Task ID            Status        Description     
    --------------------------------------------------------------------------------
    """
    
    headers = "{:<10} {:<15} {:>10}".format(*columns)
    print("-" * 80)
    print(headers)
    print("-" * 80)

### User Name and Password Management
- Create hash of plain password using salt
- varify password for user login
- convert byte to str and vice versa for storing the password in json file.

In [276]:
# Password management

# Hash password before storing to file
def hash_password(password: str) -> str:
    # Generate a salt
    salt = bcrypt.gensalt()
    
    # Hash the password with the salt
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), salt)
    
    return hashed_password

def verify_password(stored_hash: str, password: str) -> bool:
    # Check if the password matches the stored hash
    return bcrypt.checkpw(password.encode('utf-8'), stored_hash)

# Function to convert bytes to a base64 encoded string
def bytes_to_str(byte_data):
    return base64.b64encode(byte_data).decode('utf-8')

# Function to convert a base64 encoded string back to bytes
def str_to_bytes(encoded_str):
    return base64.b64decode(encoded_str.encode('utf-8'))


In [254]:
# User related functions
def update_user(filename, data):
    json = read_from_json(filename)
    idx = 0 if not json else len(json)

    json[str(idx+1)] = data
    write_to_json(filename, json)
    print('User is saved')
    return idx+1

def save_user(user_name, password):
    password = hash_password(password)
    return update_user('users.json', {"User Name": user_name, "Password": bytes_to_str(password)})
    
def get_existing_usernames():
    user_names = set()
    data = read_from_json('users.json')
    if not data: return user_names
    for idx, user in data.items():
        user_names.add(user["User Name"])
    return user_names
    
def verify_user(user_name, password):
    user_data = read_from_json('users.json')
    if not user_data: return
    for user_id, user in user_data.items():
        if user_name != user["User Name"]:continue
        password_hash = str_to_bytes(user["Password"])
        if verify_password(password_hash, password): return user_id
    return 



### User Registration and login functionalty
- Register user
- login user
- logout user

In [ ]:

# Register user
def registration():
    print("===================user registration =============")
    existing_user_names = ()
    while True:
        user_name = input("Please enter user name: ")
        if user_name not in get_existing_usernames(): break
        print(f"Username -'{user_name}' already exist, Choose different username")
    password = input("Please enter user password: ")
    if not password:
        print('Please enter password!')
    
    if user_name and password:
        user_id = save_user(user_name, password)
        print(f"You have created account successfully. Please use username-{user_name} to login")
        return user_id
    return 

def login():
    user_name = input("Please enter user name: ")
    password = input("Please enter user password: ")
    user_id = verify_user(user_name, password)
    if user_id:
        print(" Logged in")
        return user_id
    
    print('Username or password is incorrect!')
    return 

def logout():
    print("You have logged out successfully.")

### Task management
- Add Task
- Delete Task
- Mark as Completed Task
- View Taks

In [257]:
# Task management functionality
def update_task(filename, user_id, data):
    json = read_from_json(filename)
    if not json:
        json = {user_id: {}}
    if json and not json.get(user_id):
        json[user_id]= {}
        
    idx= len(json[user_id])
    data["Task Id"] = str(idx+1)
    json[user_id][str(idx+1)] = data
    write_to_json(filename, json)
    print("Task is added.")

def add_task(user_id):
    description = input("Enter the Task description:")
    task = {"Status": 'pending', "Description": description}
    update_task('tasks.json', user_id, task)

def delete_task(user_id, task_id):
    json = read_from_json('tasks.json')
    if not json.get(user_id) or not json[user_id].get(task_id):
        print("======> No Task with this Id. Ignoring..")
        return

    # delete the task with task id
    del json[user_id][task_id]
    write_to_json('tasks.json', json)
    print("Task is deleted.")

def view_tasks(user_id):
    json = read_from_json('tasks.json')
    if not json or not json.get(user_id): 
        print("======> No Task created!")
        return
    tasks = json[user_id]
    print_header(["Task ID", "Status", "Description"])
    for _, task in tasks.items():
        print("{:<10} {:<15} {:>10} ".format(
                task['Task Id'], 
                task['Status'], 
                task['Description'])
                 )
        
def mark_completed(user_id, task_id):
    json = read_from_json('tasks.json')
    if not json.get(user_id) or not json[user_id].get(task_id):
        print("======> No Task with this Id. Ignoring..")
        return
    json[user_id][task_id]['Status'] = 'Completed'
    write_to_json('tasks.json', json)
    print("Task is marked completed.")
    

### Main Menu function or Entry Point
- Display nice header for application.
- Ask user to log in.
- if user is not registered, Ask user to register.
- once logged in, Display options to choose to proceed ahead.

In [259]:
# utility function to print a nice header for Task manager application.
def banner():
    nl = '\n'
    header = '='*80
    banner = header + nl+ '|' + 'TASK MANAGER'.center(78) + '|' + nl + header
    print()
    print(banner)

# menu function to list down all the options. 
# User can choose any option to perform actions.
def menu():
    banner()
    # User to log in
    user_id = login()
    while not user_id:
        create_account = input(" Do you want to create account, type (y/n):")
        if create_account.lower() =='y':
            user_id = registration()
        else:
            user_id = login()
    
    # It will show the menu option until the user exits.
    while True:
        print()
        print("1. Add Task")
        print("2. View Tasks")
        print("3. Mark as Completed")
        print("4. Delete Task")
        print("5. Logout")

        choice = input("Enter your choice: ")

        if choice == '1':
            add_task(user_id)
        elif choice == '2':
            view_tasks(user_id)
        elif choice == '3':
            task_id = input("Please enter Task Id to mark 'Completed': ")
            mark_completed(user_id, task_id)
        elif choice == '4':
            task_id = input("Please enter Task Id to delete: ")
            delete_task(user_id, task_id)
        elif choice == '5':
            logout()
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please try again.")

In [261]:
menu()


|                                 TASK MANAGER                                 |


Please enter user name:  shailu
Please enter user password:  admin


 Logged in

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


Enter your choice:  2


--------------------------------------------------------------------------------
Task ID    Status          Description
--------------------------------------------------------------------------------
1          pending         Task1 by user 1 

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


Enter your choice:  5


You have logged out successfully.
Exiting...


In [263]:
menu()


|                                 TASK MANAGER                                 |


Please enter user name:  shailu2
Please enter user password:  admin2


 Logged in

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


Enter your choice:  2


======> No Task created!

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


...

Data has been written to tasks.json
Task is marked completed.

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


Enter your choice:  2


--------------------------------------------------------------------------------
Task ID    Status          Description
--------------------------------------------------------------------------------
1          Completed       Task by user 2 
2          pending         Second task by user2 

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


Enter your choice:  5


You have logged out successfully.
Exiting...


In [265]:
menu()


|                                 TASK MANAGER                                 |


Please enter user name:  shailu3
Please enter user password:  admin3


Username or password is incorrect!


 Do you want to create account, type (y/n): y


===================user registration =============


...

======> No Task created!

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


Enter your choice:  3
Please enter Task Id to mark 'Completed':  1


======> No Task with this Id. Ignoring..

1. Add Task
2. View Tasks
3. Mark as Completed
4. Delete Task
5. Logout


Enter your choice:  5


You have logged out successfully.
Exiting...
